In [138]:
import pandas as pd
import numpy as np
import math

In [158]:
gamesdf = pd.read_csv("../resources/preclean/clean_games.csv", parse_dates=["SEASON"])
gamesdf = gamesdf.rename(columns={"Unnamed: 0" : "CATA"})
gamesdf["GAME_DATE_EST"] = pd.to_datetime(gamesdf.GAME_DATE_EST)

gamesdf

,CATA,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,0,2022-12-22,22200477,1610612740,1610612759,2022-01-01,1610612740,126.0,0.484,0.926,...,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
1,1,2022-12-22,22200478,1610612762,1610612764,2022-01-01,1610612762,120.0,0.488,0.952,...,16.0,40.0,1610612764,112.0,0.561,0.765,0.333,20.0,37.0,1
2,2,2022-12-21,22200466,1610612739,1610612749,2022-01-01,1610612739,114.0,0.482,0.786,...,22.0,37.0,1610612749,106.0,0.470,0.682,0.433,20.0,46.0,1
3,3,2022-12-21,22200467,1610612755,1610612765,2022-01-01,1610612755,113.0,0.441,0.909,...,27.0,49.0,1610612765,93.0,0.392,0.735,0.261,15.0,46.0,1
4,4,2022-12-21,22200468,1610612737,1610612741,2022-01-01,1610612737,108.0,0.429,1.000,...,22.0,47.0,1610612741,110.0,0.500,0.773,0.292,20.0,47.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26547,26646,2014-10-06,11400007,1610612737,1610612740,2014-01-01,1610612737,93.0,0.419,0.821,...,24.0,50.0,1610612740,87.0,0.366,0.643,0.375,17.0,43.0,1
26548,26647,2014-10-06,11400004,1610612741,1610612764,2014-01-01,1610612741,81.0,0.338,0.719,...,18.0,40.0,1610612764,85.0,0.411,0.636,0.267,17.0,47.0,0
26549,26648,2014-10-06,11400005,1610612747,1610612743,2014-01-01,1610612747,98.0,0.448,0.682,...,29.0,45.0,1610612743,95.0,0.387,0.659,0.500,19.0,43.0,1
26550,26649,2014-10-05,11400002,1610612761,1610612758,2014-01-01,1610612761,99.0,0.440,0.771,...,21.0,30.0,1610612758,94.0,0.469,0.725,0.385,18.0,45.0,1


In [159]:
gamesdf["REFERENCE"] = gamesdf.GAME_ID.astype(str) + gamesdf.CATA.astype(str)
gamesdf = gamesdf.drop(["GAME_ID", "CATA"], axis=1)
gamesdf
noref_games = gamesdf.drop(["REFERENCE"], axis=1)

In [160]:
gamesdf["REFERENCE"].duplicated().sum()

0

In [161]:
detailsdf = pd.read_csv("../resources/preclean/clean_games_details.csv")
detailsdf[["MIN", "SECOND"]] = detailsdf["MIN"].str.split(":", expand=True)
detailsdf = detailsdf.drop(detailsdf[["SECOND"]], axis=1)
detailsdf = detailsdf.rename(columns={"Unnamed: 0" : "CATA"})
detailsdf["MIN"] = detailsdf["MIN"].astype(float)
detailsdf

/Users/mitch/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,CATA,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,MIN,FGM,FGA,FG_PCT,FG3M,...,FG3_PCT,FTM,FTA,FT_PCT,REB,AST,STL,BLK,TO,PTS
0,0,22200477,1610612759,1629641,Romeo Langford,18.0,1.0,1.0,1.000,0.0,...,0.000,0.0,0.0,0.000,2.0,0.0,1.0,0.0,2.0,2.0
1,1,22200477,1610612759,1631110,Jeremy Sochan,31.0,7.0,14.0,0.500,2.0,...,0.500,7.0,10.0,0.700,9.0,6.0,1.0,0.0,2.0,23.0
2,2,22200477,1610612759,1627751,Jakob Poeltl,21.0,6.0,9.0,0.667,0.0,...,0.000,1.0,1.0,1.000,4.0,1.0,1.0,0.0,2.0,13.0
3,3,22200477,1610612759,1630170,Devin Vassell,30.0,4.0,13.0,0.308,1.0,...,0.167,1.0,1.0,1.000,9.0,5.0,3.0,0.0,2.0,10.0
4,4,22200477,1610612759,1630200,Tre Jones,27.0,7.0,12.0,0.583,1.0,...,0.333,4.0,4.0,1.000,2.0,3.0,0.0,0.0,2.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558933,668623,11200005,1610612743,202706,Jordan Hamilton,NaN,4.0,9.0,0.444,3.0,...,0.500,6.0,7.0,0.857,2.0,0.0,2.0,0.0,1.0,17.0
558934,668624,11200005,1610612743,202702,Kenneth Faried,NaN,7.0,11.0,0.636,0.0,...,0.000,4.0,4.0,1.000,1.0,1.0,1.0,0.0,3.0,18.0
558935,668625,11200005,1610612743,201585,Kosta Koufos,NaN,3.0,7.0,0.429,0.0,...,0.000,0.0,0.0,0.000,8.0,0.0,1.0,0.0,0.0,6.0
558936,668626,11200005,1610612743,202389,Timofey Mozgov,NaN,1.0,1.0,1.000,0.0,...,0.000,0.0,2.0,0.000,3.0,1.0,0.0,0.0,4.0,2.0


In [162]:
detailsdf =  detailsdf.dropna()
detailsdf["UNIQUE_ID"] = detailsdf.GAME_ID.astype(str) + detailsdf.CATA.astype(str)
detailsdf = detailsdf.drop(["CATA"], axis=1)
detailsdf

/Users/mitch/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,GAME_ID,TEAM_ID,PLAYER_ID,PLAYER_NAME,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,...,FTM,FTA,FT_PCT,REB,AST,STL,BLK,TO,PTS,UNIQUE_ID
0,22200477,1610612759,1629641,Romeo Langford,18.0,1.0,1.0,1.000,0.0,0.0,...,0.0,0.0,0.000,2.0,0.0,1.0,0.0,2.0,2.0,222004770
1,22200477,1610612759,1631110,Jeremy Sochan,31.0,7.0,14.0,0.500,2.0,4.0,...,7.0,10.0,0.700,9.0,6.0,1.0,0.0,2.0,23.0,222004771
2,22200477,1610612759,1627751,Jakob Poeltl,21.0,6.0,9.0,0.667,0.0,0.0,...,1.0,1.0,1.000,4.0,1.0,1.0,0.0,2.0,13.0,222004772
3,22200477,1610612759,1630170,Devin Vassell,30.0,4.0,13.0,0.308,1.0,6.0,...,1.0,1.0,1.000,9.0,5.0,3.0,0.0,2.0,10.0,222004773
4,22200477,1610612759,1630200,Tre Jones,27.0,7.0,12.0,0.583,1.0,3.0,...,4.0,4.0,1.000,2.0,3.0,0.0,0.0,2.0,19.0,222004774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557051,11200086,1610612747,2768,Chris Duhon,8.0,1.0,3.0,0.333,0.0,1.0,...,0.0,0.0,0.000,1.0,2.0,1.0,0.0,0.0,2.0,11200086666741
557052,11200086,1610612747,202365,Devin Ebanks,15.0,2.0,5.0,0.400,0.0,2.0,...,0.0,0.0,0.000,1.0,1.0,0.0,0.0,1.0,4.0,11200086666742
557053,11200086,1610612747,2730,Dwight Howard,33.0,8.0,12.0,0.667,0.0,0.0,...,3.0,8.0,0.375,12.0,2.0,0.0,4.0,5.0,19.0,11200086666743
557054,11200086,1610612747,201975,Jodie Meeks,15.0,2.0,4.0,0.500,2.0,2.0,...,0.0,0.0,0.000,1.0,1.0,0.0,0.0,0.0,6.0,11200086666744


In [163]:
detailsdf["UNIQUE_ID"].duplicated().sum()

0

In [164]:
detailsdf.dtypes

GAME_ID          int64
TEAM_ID          int64
PLAYER_ID        int64
PLAYER_NAME     object
MIN            float64
FGM            float64
FGA            float64
FG_PCT         float64
FG3M           float64
FG3A           float64
FG3_PCT        float64
FTM            float64
FTA            float64
FT_PCT         float64
REB            float64
AST            float64
STL            float64
BLK            float64
TO             float64
PTS            float64
UNIQUE_ID       object
dtype: object

In [165]:
rankingdf = pd.read_csv("../resources/preclean/clean_ranking.csv", parse_dates=["DATE"])
rankingdf

,Unnamed: 0,TEAM_ID,CONFERENCE,TEAM,G,W,L,W_PCT,DATE
0,0,1610612743,West,Denver,30,19,11,0.633,2022-12-22
1,1,1610612763,West,Memphis,30,19,11,0.633,2022-12-22
2,2,1610612740,West,New Orleans,31,19,12,0.613,2022-12-22
3,3,1610612756,West,Phoenix,32,19,13,0.594,2022-12-22
4,4,1610612746,West,LA Clippers,33,19,14,0.576,2022-12-22
...,...,...,...,...,...,...,...,...,...
210337,210337,1610612765,East,Detroit,82,29,53,0.354,2014-09-01
210338,210338,1610612738,East,Boston,82,25,57,0.305,2014-09-01
210339,210339,1610612753,East,Orlando,82,23,59,0.280,2014-09-01
210340,210340,1610612755,East,Philadelphia,82,19,63,0.232,2014-09-01


In [166]:
rankingdf = rankingdf.rename(columns={"Unnamed: 0" : "CATA"})
rankingdf["RANK_ID"] = rankingdf.TEAM_ID.astype(str) + rankingdf.CATA.astype(str)
rankingdf = rankingdf.drop(["CATA"], axis=1)
rankingdf.head()

,TEAM_ID,CONFERENCE,TEAM,G,W,L,W_PCT,DATE,RANK_ID
0,1610612743,West,Denver,30,19,11,0.633,2022-12-22,16106127430
1,1610612763,West,Memphis,30,19,11,0.633,2022-12-22,16106127631
2,1610612740,West,New Orleans,31,19,12,0.613,2022-12-22,16106127402
3,1610612756,West,Phoenix,32,19,13,0.594,2022-12-22,16106127563
4,1610612746,West,LA Clippers,33,19,14,0.576,2022-12-22,16106127464


In [167]:
rankingdf["RANK_ID"].duplicated().sum()

0

In [168]:
rankingdf.dtypes

TEAM_ID                int64
CONFERENCE            object
TEAM                  object
G                      int64
W                      int64
L                      int64
W_PCT                float64
DATE          datetime64[ns]
RANK_ID               object
dtype: object

In [169]:
playersdf = pd.read_csv("../resources/preclean/players.csv", parse_dates=["SEASON"])
playersdf

,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON
0,Royce O'Neale,1610612762,1626220,2019-01-01
1,Bojan Bogdanovic,1610612762,202711,2019-01-01
2,Rudy Gobert,1610612762,203497,2019-01-01
3,Donovan Mitchell,1610612762,1628378,2019-01-01
4,Mike Conley,1610612762,201144,2019-01-01
...,...,...,...,...
7223,Lanny Smith,1610612758,201831,2009-01-01
7224,Warren Carter,1610612752,201999,2009-01-01
7225,Bennet Davis,1610612751,201834,2009-01-01
7226,Brian Hamilton,1610612751,201646,2009-01-01


In [170]:
playersdf["PLAYER_ID"].isnull().sum()

0

In [171]:
playersdf["PLAYER_ID"].duplicated().sum()

5459

In [172]:
playersdf["CATA"] = range(len(playersdf["PLAYER_ID"]))
playersdf["TABLE_ID"] = playersdf.PLAYER_ID.astype(str) + playersdf.CATA.astype(str)
playersdf = playersdf.drop(["CATA"], axis=1)
playersdf

,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON,TABLE_ID
0,Royce O'Neale,1610612762,1626220,2019-01-01,16262200
1,Bojan Bogdanovic,1610612762,202711,2019-01-01,2027111
2,Rudy Gobert,1610612762,203497,2019-01-01,2034972
3,Donovan Mitchell,1610612762,1628378,2019-01-01,16283783
4,Mike Conley,1610612762,201144,2019-01-01,2011444
...,...,...,...,...,...
7223,Lanny Smith,1610612758,201831,2009-01-01,2018317223
7224,Warren Carter,1610612752,201999,2009-01-01,2019997224
7225,Bennet Davis,1610612751,201834,2009-01-01,2018347225
7226,Brian Hamilton,1610612751,201646,2009-01-01,2016467226


In [173]:
playersdf["TABLE_ID"].duplicated().sum()

0

In [174]:
playersdf.dtypes

PLAYER_NAME            object
TEAM_ID                 int64
PLAYER_ID               int64
SEASON         datetime64[ns]
TABLE_ID               object
dtype: object

In [175]:
gamesdf.to_csv("../resources/games.csv", index=False)
detailsdf.to_csv("../resources/game_details.csv", index=False)
rankingdf.to_csv("../resources/ranking.csv", index=False)
playersdf.to_csv("../resources/players.csv", index=False)
noref_games.to_csv("../resources/noref_games.csv", index=False)